# Setup
This IPython notebook will walk through the steps of characterizing iModulons through the semi-automated tools in PyModulon. You will need:

* M and A matrices
* Expression data (e.g. `log_tpm_norm.csv`)
* Gene table and KEGG/GO annotations (Generated in `gene_annotation.ipynb`)
* Sample table, with a column for `project` and `condition`
* TRN file

Optional:
* iModulon table (if you already have some characterized iModulons)

In [1]:
from pymodulon.core import IcaData
from pymodulon.plotting import *
from os import path
import pandas as pd
import re
from Bio.KEGG import REST
from tqdm.notebook import tqdm

In [2]:
# Enter the location of your data here
data_dir = path.join('../..','data','processed', 'modulome')

# GO and KEGG annotations are in the 'external' folder
external_data = path.join('../..','data','external', 'modulome')
interim_data = path.join('../..','data','interim', 'modulome')

## Check your sample table (i.e. metadata file)
Your metadata file will probably have a lot of columns, most of which you may not care about. Feel free to save a secondary copy of your metadata file with only columns that seem relevant to you. The two most important columns are:
1. `project`
2. `condition`

Make sure that these columns exist in your metadata file

In [3]:
df_metadata = pd.read_csv(path.join(data_dir,'metadata_sorted.tsv'),index_col=0,sep='\t')
df_metadata[['project','condition']].head()

,project,condition
experiment,,
Control1-MSG,Control,control
Control2-MSG,Control,control
27-J1074-RNA-Cryo-MM,Batchcontrol,Batchcontrol
28-J1074-RNA-Cryo-MM,Batchcontrol,Batchcontrol
7_Plate_A_MS,Solid culture,Plate_A


In [4]:
print(df_metadata.project.notnull().all())
print(df_metadata.condition.notnull().all())

True
True


# Organize the data
We need to modify the sample order of the A.csv file because it governs the order of samples in the dashboard. We will use the `df_metadata` to do this.

In [5]:
# read A.csv
A = pd.read_csv(path.join(data_dir,'A.csv'),index_col=0)
A = A.reindex(columns=df_metadata['sample_id'])
A.to_csv(path.join(data_dir,'A_ordered.csv'))

In [6]:
# read log_tpm_norm.csv
log_tpm = pd.read_csv(path.join(data_dir,'log_tpm_norm.csv'),index_col=0)
log_tpm = log_tpm.reindex(columns=df_metadata['sample_id'])
log_tpm.to_csv(path.join(data_dir,'log_tpm_norm_ordered.csv'))

## Load the data
You're now ready to load your IcaData object!

In [5]:
ica_data = IcaData(M = path.join(data_dir,'M.csv'),
                   A = path.join(data_dir,'A_ordered.csv'),
                   X = path.join(data_dir,'log_tpm_norm_ordered.csv'),
                   gene_table = path.join(data_dir,'gene_info6.csv'),
                   sample_table = path.join(data_dir,'metadata_sorted.tsv'),
                   threshold_method='kmeans')

# Functional iModulons

GO annotations and KEGG pathways/modules were generated in the 1_create_the_gene_table.ipynb notebook. Enrichments will be calculated in this notebook, and further curated in the 3_manual_iModulon_curation notebook.

## GO Enrichments

First load the Gene Ontology annotations

In [6]:
DF_GO = pd.read_csv(path.join(external_data,'GO_annotations_curated.csv'),index_col=0)
DF_GO.head()

,locus_tag,GOs
54,XNR_RS00270,mannosyltransferase activity
55,XNR_RS00270,molecular_function
56,XNR_RS00270,catalytic activity
57,XNR_RS00270,dolichyl-phosphate beta-D-mannosyltransferase ...
58,XNR_RS00270,cellular_component


In [7]:
# Change 'locus_tag' to 'gene_id'
DF_GO = DF_GO.rename(columns={'locus_tag':'gene_id'})

In [8]:
DF_GO_enrich = ica_data.compute_annotation_enrichment(DF_GO,'GOs')

In [9]:
DF_GO_enrich

,imodulon,GOs,pvalue,qvalue,precision,recall,f1score,TP,target_set_size,imodulon_size
0,0,"N-alpha,N-alpha,N-alpha-trimethyl-L-histidine ...",0.000080,0.016441,0.037736,1.000000,0.072727,2.0,2.0,53.0
1,0,amino-acid betaine biosynthetic process,0.000080,0.016441,0.037736,1.000000,0.072727,2.0,2.0,53.0
2,0,amino-acid betaine metabolic process,0.000080,0.016441,0.037736,1.000000,0.072727,2.0,2.0,53.0
3,0,ammonium ion metabolic process,0.000080,0.016441,0.037736,1.000000,0.072727,2.0,2.0,53.0
4,0,cellular modified histidine biosynthetic process,0.000080,0.016441,0.037736,1.000000,0.072727,2.0,2.0,53.0
...,...,...,...,...,...,...,...,...,...,...
243,71,ncRNA processing,0.002176,0.051520,0.036364,0.133333,0.057143,4.0,30.0,110.0
244,71,rRNA processing,0.002453,0.057121,0.027273,0.200000,0.048000,3.0,15.0,110.0
245,71,RNA processing,0.002463,0.057121,0.036364,0.129032,0.056738,4.0,31.0,110.0
246,71,RNA metabolic process,0.002926,0.067171,0.045455,0.094340,0.061350,5.0,53.0,110.0


## KEGG Enrichments

### Load KEGG mapping
The `kegg_mapping.csv` file contains KEGG orthologies, pathways, modules, and reactions. Only pathways and modules are relevant to iModulon characterization.

In [10]:
DF_KEGG = pd.read_csv(path.join(external_data,'kegg_mapping.csv'),index_col=0)
print(DF_KEGG.database.unique())
DF_KEGG.head()

['KEGG_ko' 'KEGG_Pathway' 'KEGG_Module' 'KEGG_Reaction']


,gene_id,database,kegg_id
0,XNR_RS30570,KEGG_ko,-
1,XNR_RS00010,KEGG_ko,-
2,XNR_RS00015,KEGG_ko,-
3,XNR_RS00020,KEGG_ko,-
4,XNR_RS00025,KEGG_ko,-


In [11]:
kegg_pathways = DF_KEGG[DF_KEGG.database == 'KEGG_Pathway']
kegg_modules = DF_KEGG[DF_KEGG.database == 'KEGG_Module']

### Perform enrichment
Uses the `compute_annotation_enrichment` function

In [12]:
DF_pathway_enrich = ica_data.compute_annotation_enrichment(kegg_pathways,'kegg_id')
DF_module_enrich = ica_data.compute_annotation_enrichment(kegg_modules,'kegg_id')

In [13]:
DF_pathway_enrich.head()

,imodulon,kegg_id,pvalue,qvalue,precision,recall,f1score,TP,target_set_size,imodulon_size
0,0,map00020,0.000345,0.083798,0.075472,0.105263,0.087912,4.0,38.0,53.0
1,5,map00525,0.000033,0.005310,0.100000,0.666667,0.173913,2.0,3.0,20.0
2,5,map01053,0.000066,0.005310,0.100000,0.500000,0.166667,2.0,4.0,20.0
3,5,map01130,0.000055,0.005310,0.350000,0.021875,0.041176,7.0,320.0,20.0
4,5,map00405,0.000303,0.014986,0.100000,0.250000,0.142857,2.0,8.0,20.0


In [14]:
DF_module_enrich.head()

,imodulon,kegg_id,pvalue,qvalue,precision,recall,f1score,TP,target_set_size,imodulon_size
0,0,M00011,0.000026,0.003509,0.075472,0.200000,0.109589,4.0,20.0,53.0
1,0,M00149,0.000024,0.003509,0.056604,0.428571,0.100000,3.0,7.0,53.0
2,0,M00009,0.000064,0.005907,0.075472,0.160000,0.102564,4.0,25.0,53.0
3,0,M00376,0.000185,0.012721,0.056604,0.230769,0.090909,3.0,13.0,53.0
4,0,M00173,0.000603,0.027655,0.056604,0.157895,0.083333,3.0,19.0,53.0


### Convert KEGG IDs to human-readable names

In [17]:
import urllib.error

for idx,key in tqdm(DF_pathway_enrich.kegg_id.items(),total=len(DF_pathway_enrich)):
    try:
        text = REST.kegg_find('pathway',key).read()
        name = re.search('\t(.*)\n',text).group(1)
        DF_pathway_enrich.loc[idx,'pathway_name'] = name
    except (AttributeError, urllib.error.HTTPError):
        print(f"Bad KEGG ID: {key}")
        DF_pathway_enrich.loc[idx,'pathway_name'] = None
    
for idx,key in tqdm(DF_module_enrich.kegg_id.items(),total=len(DF_module_enrich)):
    try:
        text = REST.kegg_find('module',key).read()
        name = re.search('\t(.*)\n',text).group(1)
        DF_module_enrich.loc[idx,'module_name'] = name
    except (AttributeError, urllib.error.HTTPError):
        print(f"Bad KEGG ID: {key}")
        DF_module_enrich.loc[idx,'module_name'] = None

  0%|          | 0/192 [00:00<?, ?it/s]

Bad KEGG ID: map01130
Bad KEGG ID: -
Bad KEGG ID: -
Bad KEGG ID: map01130
Bad KEGG ID: map00281
Bad KEGG ID: -
Bad KEGG ID: map01130
Bad KEGG ID: map00072
Bad KEGG ID: map00281


  0%|          | 0/124 [00:00<?, ?it/s]

Bad KEGG ID: M00254
Bad KEGG ID: M00479
Bad KEGG ID: M00258
Bad KEGG ID: M00188
Bad KEGG ID: M00207
Bad KEGG ID: M00242
Bad KEGG ID: M00244
Bad KEGG ID: M00178
Bad KEGG ID: M00240
Bad KEGG ID: M00200
Bad KEGG ID: M00207
Bad KEGG ID: M00236
Bad KEGG ID: M00435
Bad KEGG ID: M00436
Bad KEGG ID: M00239
Bad KEGG ID: M00222
Bad KEGG ID: M00178
Bad KEGG ID: M00216
Bad KEGG ID: M00460
Bad KEGG ID: M00491
Bad KEGG ID: M00454
Bad KEGG ID: M00670
Bad KEGG ID: M00210
Bad KEGG ID: M00669
Bad KEGG ID: M00323
Bad KEGG ID: M00167
Bad KEGG ID: M00212
Bad KEGG ID: M00215
Bad KEGG ID: M00323
Bad KEGG ID: M00216
Bad KEGG ID: M00166
Bad KEGG ID: M00233
Bad KEGG ID: M00239
Bad KEGG ID: M00237
Bad KEGG ID: M00240
Bad KEGG ID: M00206
Bad KEGG ID: M00207
Bad KEGG ID: M00191
Bad KEGG ID: M00439
Bad KEGG ID: M00239
Bad KEGG ID: M00178
Bad KEGG ID: M00179
Bad KEGG ID: M00183
Bad KEGG ID: M00439
Bad KEGG ID: M00239
Bad KEGG ID: M00237
Bad KEGG ID: M00236
Bad KEGG ID: M00205
Bad KEGG ID: M00236
Bad KEGG ID: M00205


In [18]:
DF_pathway_enrich.head()

,imodulon,kegg_id,pvalue,qvalue,precision,recall,f1score,TP,target_set_size,imodulon_size,pathway_name
0,0,map00020,0.000345,0.083798,0.075472,0.105263,0.087912,4.0,38.0,53.0,Citrate cycle (TCA cycle)
1,5,map00525,0.000033,0.005310,0.100000,0.666667,0.173913,2.0,3.0,20.0,Acarbose and validamycin biosynthesis
2,5,map01053,0.000066,0.005310,0.100000,0.500000,0.166667,2.0,4.0,20.0,Biosynthesis of siderophore group nonribosomal...
3,5,map01130,0.000055,0.005310,0.350000,0.021875,0.041176,7.0,320.0,20.0,None
4,5,map00405,0.000303,0.014986,0.100000,0.250000,0.142857,2.0,8.0,20.0,Phenazine biosynthesis


In [19]:
DF_module_enrich.head()

,imodulon,kegg_id,pvalue,qvalue,precision,recall,f1score,TP,target_set_size,imodulon_size,module_name
0,0,M00011,0.000026,0.003509,0.075472,0.200000,0.109589,4.0,20.0,53.0,"Citrate cycle, second carbon oxidation, 2-oxog..."
1,0,M00149,0.000024,0.003509,0.056604,0.428571,0.100000,3.0,7.0,53.0,"Succinate dehydrogenase, prokaryotes"
2,0,M00009,0.000064,0.005907,0.075472,0.160000,0.102564,4.0,25.0,53.0,"Citrate cycle (TCA cycle, Krebs cycle)"
3,0,M00376,0.000185,0.012721,0.056604,0.230769,0.090909,3.0,13.0,53.0,3-Hydroxypropionate bi-cycle
4,0,M00173,0.000603,0.027655,0.056604,0.157895,0.083333,3.0,19.0,53.0,Reductive citrate cycle (Arnon-Buchanan cycle)


## Save files

In [20]:
DF_GO_enrich['source'] = 'GO'
# DF_pathway_enrich['source'] = 'KEGG pathways'
# DF_module_enrich['source'] = 'KEGG modules'
# DF_subti_enrich['source'] = 'SubtiWiki'

DF_GO_enrich.rename({'gene_ontology':'annotation'},axis=1, inplace=True)
# DF_pathway_enrich.rename({'kegg_id':'annotation'},axis=1, inplace=True)
# DF_module_enrich.rename({'kegg_id':'annotation'},axis=1, inplace=True)
# DF_subti_enrich.rename({'value':'annotation'},axis=1, inplace=True)

DF_enrichments = pd.concat([DF_GO_enrich, DF_pathway_enrich, DF_module_enrich])
DF_enrichments.to_csv(path.join(data_dir,'functional_enrichments.csv'))

# Check for single gene iModulons

Some iModulons are dominated by a single, high-coefficient gene. These iModulons may result from:
1. Overdecomposition of the dataset to identify noisy genes
1. Artificial knock-out of single genes
1. Regulons with only one target gene

No matter what causes these iModulons, it is important to be aware of them. The find_single_gene_imodulons function identifies iModulons that are likely dominated by a single gene.

The iModulons identified by ``find_single_gene_imodulons`` may contain more than one gene, since a threshold-agnostic method is used to identify these iModulons.

In [21]:
sg_imods = ica_data.find_single_gene_imodulons(save=True)
len(sg_imods)

0

In [22]:
for i,mod in enumerate(sg_imods):
    ica_data.rename_imodulons({mod:'SG_'+str(i+1)})

In [37]:
ica_data.imodulon_table[ica_data.imodulon_table.single_gene == True]

AttributeError: 'DataFrame' object has no attribute 'single_gene'

In [15]:
ica_data.view_imodulon(12)

,gene_weight,gene_name,eggNOG_OGs,Description,GOs,EC,KEGG_ko,KEGG_Pathway,KEGG_Module,KEGG_Reaction,...,BiGG_Reaction,PFAMs,accession,old_locus_tag,start,end,strand,gene_product,COG,operon
XNR_RS29710,0.091595,XNR_RS29710,"COG2172@1|root,COG2172@2|Bacteria,2IMCZ@201174...",Histidine kinase-like ATPase domain,-,-,-,-,-,-,...,-,HATPase_c_2,NC_020990.1,NaN,2376752,2377186,+,ATP-binding protein,Signal transduction mechanisms,Op1487
XNR_RS12550,0.169050,XNR_RS12550,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NC_020990.1,XNR_2530,2910742,2910921,+,hypothetical protein,No COG Annotation,Op1883
XNR_RS12555,0.174173,XNR_RS12555,"2C2M0@1|root,2ZTSU@2|Bacteria,2GR3G@201174|Act...",-,-,-,-,-,-,-,...,-,-,NC_020990.1,XNR_2531,2911012,2911260,+,DUF6284 family protein,No COG Annotation,Op1884
XNR_RS12560,0.187321,XNR_RS12560,"2DDGX@1|root,2ZI1K@2|Bacteria,2HSAS@201174|Act...",-,-,-,-,-,-,-,...,-,-,NC_020990.1,XNR_2532,2911293,2912288,+,hypothetical protein,No COG Annotation,Op1884
XNR_RS12565,0.173812,XNR_RS12565,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NC_020990.1,NaN,2912285,2912615,+,hypothetical protein,No COG Annotation,Op1884
XNR_RS12570,0.172359,XNR_RS12570,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NC_020990.1,XNR_2534,2912612,2912908,+,hypothetical protein,No COG Annotation,Op1884
XNR_RS12575,0.157271,XNR_RS12575,"COG1051@1|root,COG1051@2|Bacteria,2IK42@201174...",Belongs to the Nudix hydrolase family,-,3.6.1.55,ko:K03574,-,-,-,...,-,"ATP_bind_2,NUDIX",NC_020990.1,XNR_2535,2913048,2913446,+,NUDIX hydrolase,Nucleotide transport and metabolism,Op1885
XNR_RS12580,0.155842,XNR_RS12580,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NC_020990.1,XNR_2536,2913449,2913664,+,hypothetical protein,No COG Annotation,Op1885
XNR_RS12595,0.169764,XNR_RS12595,"2CIKG@1|root,2ZS5S@2|Bacteria,2H230@201174|Act...",-,-,-,-,-,-,-,...,-,-,NC_020990.1,XNR_2539,2914615,2916195,+,hypothetical protein,No COG Annotation,Op1887
XNR_RS12600,0.150063,XNR_RS12600,"COG1674@1|root,COG1674@2|Bacteria,2I9XY@201174...",ftsk spoiiie,-,-,-,-,-,-,...,-,FtsK_SpoIIIE,NC_020990.1,XNR_2540,2916195,2918303,+,hypothetical protein,"Cell cycle control, cell division, chromosome ...",Op1887


# Save iModulon object

In [16]:
from pymodulon.util import explained_variance
from pymodulon.io import *

In [17]:
# Add iModulon sizes and explained variance
for im in ica_data.imodulon_names:
    ica_data.imodulon_table.loc[im,'imodulon_size'] = len(ica_data.view_imodulon(im))
    ica_data.imodulon_table.loc[im,'explained_variance'] = explained_variance(ica_data,imodulons=im)

In [18]:
ica_data.imodulon_table.head()

,imodulon_size,explained_variance
0,53.0,0.026971
1,26.0,0.004914
2,20.0,0.001916
3,7.0,0.002046
4,4.0,0.002665


This will save your iModulon table, your thresholds, and any other information stored in the ica_data object.

In [27]:
save_to_json(ica_data, path.join('../..','data','interim','modulome','salb_raw.json.gz'))

If you prefer to view and edit your iModulon table in excel, save it as a CSV and reload the iModulon as before

In [28]:
ica_data.imodulon_table.to_csv(path.join('../..','data','interim','modulome','imodulon_table_raw.csv'))

In [30]:
# Read the CSV file containing the imodulon names
imodulon_names_df = pd.read_csv(path.join(interim_data, 'imodulon_table_raw.csv'))
# Extract the names into a list
imodulon_names = imodulon_names_df['Unnamed: 0'].tolist()
all_genes = []  # List to store all genes from each imodulon

for imodulon_name in imodulon_names:
    imodulome_data = ica_data.view_imodulon(imodulon_name)
    genes = imodulome_data['gene_name'].tolist()
    for gene in genes:
        all_genes.append((gene, imodulon_name))

# Convert the collected gene data to a pandas DataFrame
df = pd.DataFrame(all_genes, columns=['Gene', 'Imodulon'])

# Save the DataFrame to a CSV file
df.to_csv(path.join(data_dir,'genes_with_imodulon.csv'), index=False)

In [31]:
imodulon_names_df

,Unnamed: 0,imodulon_size,explained_variance
0,0,53.0,0.026971
1,1,26.0,0.004914
2,2,20.0,0.001916
3,3,7.0,0.002046
4,4,4.0,0.002665
...,...,...,...
73,73,134.0,0.003439
74,74,27.0,0.004961
75,75,89.0,0.029237
76,76,119.0,0.003165
